In [2]:
import pandas as pd
import time
from datasets import load_dataset
data = load_dataset("stanfordnlp/imdb")

Lets see what kind of data I am working with here and see if I can get it into a pandas dataframe.

In [3]:
display(type(data))
display(data)
train_data = pd.DataFrame(data['train'])
test_data = pd.DataFrame(data['test'])
# display("Training Data: ", train_data.head())
# display("Testing Data: ", test_data.head())

datasets.dataset_dict.DatasetDict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Check distribution of positive/negative labels, confirm what is in the instructions

In [3]:
# display(train_data['label'].value_counts())

Even data distribution, no SMOTE algorithm

check null

In [4]:
# print(train_data.isnull().sum())

No nulls is good nulls

In [5]:
# train_data['length'] = train_data['text'].apply(len)
# print(train_data['length'].describe())

In [6]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.histplot(train_data['length'], bins=30)
# plt.title('Distribution of Review Lengths')
# plt.xlabel('Length of Review')
# plt.ylabel('Frequency')
# plt.show()

In [7]:
# from wordcloud import WordCloud

# positive_reviews = ' '.join(train_data[train_data['label'] == 1]['text'])
# negative_reviews = ' '.join(train_data[train_data['label'] == 0]['text'])

# wordcloud_positive = WordCloud(width=800, height=400).generate(positive_reviews)
# wordcloud_negative = WordCloud(width=800, height=400).generate(negative_reviews)

# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.imshow(wordcloud_positive, interpolation='bilinear')
# plt.axis('off')
# plt.title('Positive Reviews')

# plt.subplot(1, 2, 2)
# plt.imshow(wordcloud_negative, interpolation='bilinear')
# plt.axis('off')
# plt.title('Negative Reviews')

# plt.show()

did the wordcloud becaus i like them, not because they will be useful here, but I need to see what br is. Most likely a newline break tag

In [4]:
pd.set_option('display.max_colwidth', None) # https://stackoverflow.com/questions/25351968/how-can-i-display-full-non-truncated-dataframe-information-in-html-when-conver
display("Training Data: ", train_data.head(1))

'Training Data: '

,text,label
0,"I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered ""controversial"" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot.",0


nailed it, remove them from training and testing

In [5]:
train_data['text'] = train_data['text'].apply(lambda x: x.replace("<br />"," "))
test_data['text'] = test_data['text'].apply(lambda x: x.replace("<br />"," "))
display("Training Data: ", train_data.head(1))

'Training Data: '

,text,label
0,"I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered ""controversial"" I really had to see this for myself. The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films. I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot.",0


redo the wordcloud

In [10]:
# positive_reviews = ' '.join(train_data[train_data['label'] == 1]['text'])
# negative_reviews = ' '.join(train_data[train_data['label'] == 0]['text'])

# wordcloud_positive = WordCloud(width=800, height=400).generate(positive_reviews)
# wordcloud_negative = WordCloud(width=800, height=400).generate(negative_reviews)

# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.imshow(wordcloud_positive, interpolation='bilinear')
# plt.axis('off')
# plt.title('Positive Reviews')

# plt.subplot(1, 2, 2)
# plt.imshow(wordcloud_negative, interpolation='bilinear')
# plt.axis('off')
# plt.title('Negative Reviews')

# plt.show()

confident its good now, lets get into it

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

set up data

In [7]:
vectorized = TfidfVectorizer()

# training data
X_train = vectorized.fit_transform(train_data['text'])
y_train = train_data['label']

# testing data
X_test = vectorized.transform(test_data['text'])
y_test = test_data['label']

grid search svm parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}

# Create a GridSearchCV object
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy') # https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.GridSearchCV.html

# Fit it to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and estimator
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best estimator: {grid_search.best_estimator_}")

# Use the best estimator to make predictions
best_svm = grid_search.best_estimator_

svm_start_pred = time.time()
svm_y_pred = best_svm.predict(X_test)
svm_end_pred = time.time()
svm_pred_time = round((svm_end_pred - svm_start_pred) / 60, 2)
print("Time to predict with best SVM: ", svm_pred_time, " minutes.")

print(f'Accuracy: {accuracy_score(y_test, svm_y_pred)}')
print(classification_report(y_test, svm_y_pred))

In [13]:
svm_start_train = time.time()
svm = SVC(kernel = 'linear') # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
svm.fit(X_train, y_train)
svm_end_train = time.time()
svm_fit_time = round((svm_end_train - svm_start_train) / 60,2)
print("Time to train SVM: ", svm_fit_time, " minutes.")

svm_start_pred = time.time()
svm_y_pred = svm.predict(X_test)
svm_end_pred = time.tim()
svm_pred_time = round((svm_end_pred - svm_start_pred) / 60,2)
print("Time to predict SVM: ", svm_pred_time, " minutes.")

print(f'Accuracy: {accuracy_score(y_test, svm_y_pred)}')
print(classification_report(y_test, svm_y_pred))

Time to train SVM:  14.95  minutes.


do it for NB using the same organization

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [26]:
nb_start_train = time.time()
nb = MultinomialNB() # https://scikit-learn.org/0.24/modules/generated/sklearn.naive_bayes.MultinomialNB.html
nb.fit(X_train, y_train)
nb_end_train = time.time()
nb_fit_time = round((nb_end_train - nb_start_train) / 60,2)
print("Time to predict NB: ", nb_fit_time, " minutes.")

nb_start_test = time.time()
nb_y_pred = nb.predict(X_test)
nb_end_test = time.time()
nb_pred_time = round((nb_end_test - nb_start_test) / 60,2)
print("Time to fit NB: ", nb_pred_time, " minutes.")

print(f'Accuracy: {accuracy_score(y_test, nb_y_pred)}')
print(classification_report(y_test, nb_y_pred))

Time to predict NB:  0.0  minutes.
Time to fit NB:  0.0  minutes.
Accuracy: 0.83024
              precision    recall  f1-score   support

           0       0.80      0.89      0.84     12500
           1       0.87      0.77      0.82     12500

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
weighted avg       0.83      0.83      0.83     25000



In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
lr_start_train = time.time()
lr = LogisticRegression() # https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html
lr.fit(X_train, y_train)
lr_end_train = time.time()
lr_fit_time = round((lr_end_train - lr_start_train) / 60,2)
print("Time to predict NB: ", lr_fit_time, " minutes.")

lr_start_test = time.time()
lr_y_pred = lr.predict(X_test)
lr_end_test = time.time()
lr_pred_time = round((lr_end_test - lr_start_test) / 60,2)
print("Time to fit NB: ", lr_pred_time, " minutes.")

print(f'Accuracy: {accuracy_score(y_test, lr_y_pred)}')
print(classification_report(y_test, lr_y_pred))

Time to predict NB:  0.05  minutes.
Time to fit NB:  0.0  minutes.
Accuracy: 0.88284
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



possible CNN code

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense


# Encode labels
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['label'])

# Tokenize the text data
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))  # For binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test accuracy: {accuracy}')
